In [3]:
# Version 25.10.23
# The model is from Wrobel_Nowak_2022, open modality: https://huggingface.co/enelpol/evalatin2022-pos-open
# This script takes an input_sentence (string) and returns a POS-tag for every word as a .csv/txt file (columns tab separated)

# Necessary installations: 
    #pip install torch transformers
    #pip install torch torchvision
    #pip install sentencepiece
    #pip install jupyter

from transformers import XLMRobertaForTokenClassification, XLMRobertaTokenizer 
import torch
import string
from collections import Counter

# Load the input
input_sentence = "iam consules erant C. Plautius iterum, L. Aemilius Mamercinus, cum Setini Norbanique Romam nuntii defectionis Privernatium cum querimoniis acceptae cladis venerunt. Volscorum item exercitum duce Antiati populo consedisse ad Satricum adlatum est. utrumque bellum Plautio sorte evenit. prius ad Privernum profectus extemplo acie conflixit. haud magno certamine devicti hostes; oppidum captum redditumque Privernatibus praesidio valido inposito; agri partes duae ademptae. inde victor exercitus Satricum contra Antiatis ductus. ibi magna utrimque caede atrox proelium fuit; et cum tempestas eos neutro inclinata spe dimicantes diremisset, Romani, nihil eo certamine tam ambiguo fessi, in posterum diem proelium parant. Volscis recensentibus, quos viros in acie amisissent, haudquaquam idem animus ad iterandum periculum fuit; nocte pro victis Antium agmine trepido sauciis ac parte inpedimentorum relicta abierunt. armorum magna vis cum inter caesa hostium corpora tum in castris inventa est. ea Luae matri dare se consul dixit finesque hostium usque ad oram maritumam est depopulatus. alteri consuli Aemilio ingresso Sabellum agrum non castra Samnitium, non legiones usquam oppositae. ferro ignique vastantem agros legati Samnitium pacem orantes adeunt; a quo reiecti ad senatum potestate facta dicendi positis ferocibus animis pacem sibi ab Romanis bellique ius adversus Sidicinos petierunt: quae se eo iustius petere, quod et in amicitiam populi Romani secundis suis rebus, non adversis, ut Campani, venissent et adversus Sidicinos sumerent arma, suos semper hostes, populi Romani numquam amicos, qui nec, ut Samnites, in pace amicitiam nec, ut Campani, auxilium in bello petissent, nec in fide populi Romani nec in dicione essent. cum de postulatis Samnitium T. Aemilius praetor senatum consuluisset reddendumque iis foedus patres censuissent, praetor Samnitibus respondit: nec, quo minus perpetua cum eis amicitia esset, per populum Romanum stetisse, nec contradici, quin, quoniam ipsos belli culpa sua contracti taedium ceperit, amicitia de integro reconcilietur; quod ad Sidicinos attineat, nihil intercedi, quo minus Samniti populo pacis bellique liberum arbitrium sit. foedere icto cum domum revertissent, extemplo inde exercitus Romanus deductus annuo stipendio et trium mensum frumento accepto, quod pepigerat consul, ut tempus indutiis daret, quoad legati redissent. Samnites copiis iisdem, quibus usi adversus Romanum bellum fuerant, contra Sidicinos profecti haud in dubia spe erant mature urbis hostium potiundae; tum ab Sidicinis deditio prius ad Romanos coepta fieri est; dein, postquam patres ut seram eam ultimaque tandem necessitate expressam aspernabantur, ad Latinos iam sua sponte in arma motos facta est.  ne Campani quidem—adeo iniuriae Samnitium quam beneficii Romanorum memoria praesentior erat—his se armis abstinuere.ex his tot populis unus ingens exercitus duce Latino fines Samnitium ingressus plus populationibus quam proeliis cladium fecit; et quamquam superiores certaminibus Latini erant, haud inviti, ne saepius dimicandum foret, agro hostium excessere. id spatium Samnitibus datum est Romam legatos mittendi; qui cum adissent senatum, conquesti eadem se foederatos pati, quae hostes essent passi, precibus infimis petiere, ut satis ducerent Romani victoriam quam Samnitibus ex Campano Sidicinoque hoste eripuisse, ne vinci etiam se ab ignavissimis populis sinerent; Latinos Campanosque, si sub dicione populi Romani essent, pro imperio arcerent Samniti agro, sin imperium abnuerent, armis coercerent. adversus haec responsum anceps datum, quia fateri pigebat in potestate sua Latinos iam non esse timebantque, ne arcendo abalienarent: Campanorum aliam condicionem esse, qui non foedere, sed per deditionem in fidem venissent; itaque Campanos, seu velint seu nolint, quieturos; in foedere Latino nihil esse, quo bellare, cum quibus ipsi velint, prohibeantur."
# Set output name
output_name = "output_name.csv"

# Load the model and tokenizer
model_path = "."
tokenizer = XLMRobertaTokenizer.from_pretrained(model_path)
model = XLMRobertaForTokenClassification.from_pretrained(model_path)
model.eval()

# Preprocessing function; punctuation and asterisks
def preprocess_text(text):
    # Add spaces around punctuation so they're treated as separate tokens
    for punct in string.punctuation:
        text = text.replace(punct, f' {punct} ')

    # Add spaces before and after words surrounded by double asterisks. -> Our citation detection scripts mark citations with asterisks.
    text = text.replace('**', ' ')
    
    return text

# Merging function after subword tokenization to get whole words again
def group_subwords_to_words(tokenized_sentence):
    grouped_tokens = []
    current_word = ""
    for token in tokenized_sentence:
        if token.startswith("▁"):  # This indicates a new word in SentencePiece tokenization.
            if current_word:
                grouped_tokens.append(current_word)
            current_word = token[1:]  # skip the "▁" character
        else:
            current_word += token
    if current_word:
        grouped_tokens.append(current_word)
    return grouped_tokens

# Tokenization
tokens = tokenizer.tokenize(preprocess_text(input_sentence))
# Regroup the subword tokens
grouped_tokens = group_subwords_to_words(tokens)

# Ascertain the correct POS tags for the regrouped tokens, majority voting
predicted_indices = []
for word in grouped_tokens:
    # Tokenize each word again to get the input format for the model
    inputs = tokenizer(word, return_tensors="pt", truncation=True, padding=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
        # Getting the predicted indices for all subword tokens of the word
        word_predicted_indices = torch.argmax(outputs.logits, dim=2).squeeze().tolist()

    # Majority voting: Choose the most frequently predicted index for the word
    most_common_index, _ = Counter(word_predicted_indices).most_common(1)[0]
    predicted_indices.append(most_common_index)

# Convert numbers into tags
id2label = {
    0: "", 1: "ADJ", 2: "ADP", 3: "ADV", 4: "AUX", 5: "CCONJ", 6: "DET", 7: "INTJ", 8: "NOUN", 9: "NUM", 10: "PART", 11: "PRON", 12: "PROPN", 13: "PUNCT", 14: "SCONJ", 15: "VERB", 16: "X", 17: "O"}

predicted_tags = [id2label[id] for id in predicted_indices]


with open(output_name, "w", encoding = "utf-8") as file:
    for item, value in zip(grouped_tokens, predicted_tags):
        file.write(f"{item}\t{value}\n")